<a href="https://colab.research.google.com/github/crazy85128x/ML0930/blob/master/nlp_sentiment_average.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz", 
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
    extract=True,
)

84131840/84125825 [==============================] - 5s 0us/step


In [2]:
import glob
# glob.glob("/root/.keras/datasets/aclImdb/train/pos/*")
import os
import pandas as pd
def read(path):
    with open(path, "r", encoding="utf-8") as f:
        content = f.read()
    return content

def get_data(t):
    dn = os.path.dirname(dataset)
    pattern = os.path.join(dn, "aclImdb", t, "pos", "*.txt")
    pos = glob.glob(pattern)
    pattern = os.path.join(dn, "aclImdb", t, "neg", "*.txt")
    neg = glob.glob(pattern)
    sentiments = [1] * len(pos) + [0] * len(neg)
    contents = map(read, pos + neg)
    df = pd.DataFrame({
        "contents":contents,
        "sentiment":sentiments
    })
    return df

train_df = get_data("train")
test_df = get_data("test")
test_df

,contents,sentiment
0,"It kept me on the edge of my seat. True, the s...",1
1,Born in 1946 I was about eight years old when ...,1
2,In the 60's Cleveland television audiences cou...,1
3,I think it is a brilliant show with cool talki...,1
4,John Wayne's first starring role just blew me ...,1
...,...,...
24995,I am not a big fan of the Spielberg/Cruise ver...,0
24996,The director infuses this film with false dept...,0
24997,"As a flying and war movie buff, this ranks at ...",0
24998,I rented this movie under the impression that ...,0


In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
tok = Tokenizer(num_words=3000)
# fit_on_texts: fit_transform 的 fit
tok.fit_on_texts(train_df["contents"])
# seq: 把所有單字換成數字的序列
x_train_seq = tok.texts_to_sequences(train_df["contents"])
x_test_seq = tok.texts_to_sequences(test_df["contents"])

In [4]:
# tok.word_index
tok.index_word[2]

'and'

In [5]:
# pd.DataFrame(train_df_seq)
# truncating: pre 12345 -> 45 post 12345 -> 12
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train_pad = pad_sequences(x_train_seq, 512)
x_test_pad = pad_sequences(x_test_seq, 512)
pd.DataFrame(x_test_pad)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,8,3,1161,12,1,17,6,21,1867,14,45,12,68,3,2178,15,49,19,280,54,39,1029,8,11,28,40,3,706,487,178,5,121,86,11,62,627,277,22,377,146
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,13,31,666,2041,1532,1,1437,606,37,3,247,8,58,702,9,13,37,316,32,776,877,9,90,3,524,133,12,73,430,276,54,28,77,166,11,730,18,9,1674,69
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,728,196,1,2,1,1393,1,106,13,1,29,55,115,265,1016,431,53,5,12,55,2,125,71,256,234,1558,6,305,7,7,92,171,48,78,10,121,143,61,3,504
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,4,792,23,134,22,200,64,86,9,492,8,1,1844,162,10,101,11,6,28,4,1,115,245,284,2,10,481,383,11,5,29,346,34,37,369,7,7,15,58,1911
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,97,1,254,248,6,32,1,995,1489,1847,23,2437,18,1,19,14,3,223,6,21,10,188,101,4,157,422,12,67,1556,1,191,8,3,278,4,578,476,20,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,10,822,12,230,2315,50,71,3,644,1755,8,1,624,485,307,35,5,10,479,10,25,5,137,16,1,30,219,9,405,1722,99,3,321,30,1,1710,1064,1214,1688,2335
24996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,169,2752,2,26,57,8,108,47,23,197,131,104,99,1,290,1467,109,12,109,47,1098,364,2,149,20,1464,2,5,984,3,582,134,995,180,72,190,15,2484,8,110
24997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2,12,1,84,81,33,23,266,5,27,162,119,11,91,217,14,45,33,801,5,898,8,14,108,2869,14,611,47,23,108,49,1544,105,2,11,6,21,28,4,95
24998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,299,6,48,163,11,17,287,2739,7,7,1,315,941,1468,3,1234,646,80,1,174,2,1442,1919,4,1419,7,7,511,344,1,4,58,2037,177,5,11,52,558,4,55


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers import Dense, Flatten, Dropout

layers = [
  # 1~3000(token) + 0(padding)
  # params: 3001 * 128
  Embedding(3001, 128, mask_zero=True, input_length=512),
  GlobalAveragePooling1D(),
  Dense(2, activation="softmax")  
]
model = Sequential(layers)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 512, 128)          384128    
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 2)                 258       
Total params: 384,386
Trainable params: 384,386
Non-trainable params: 0
_________________________________________________________________


In [7]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
model.compile(loss=SparseCategoricalCrossentropy(),
       optimizer=Adam(),
       metrics=["accuracy"])

In [8]:
y_train = train_df["sentiment"]
y_test = test_df["sentiment"]

In [9]:
# epoch: 整份資料要看幾遍
# 60000筆, 2epoch, 200batch -> 60000 * 2 / 200次調整
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
callbacks = [
    ModelCheckpoint("model.h5", save_best_only=True),
    EarlyStopping(patience=5, restore_best_weights=True)
]
model.fit(x_train_pad, 
     y_train,
     batch_size=100,
     epochs=50,
     validation_split=0.1,
     callbacks=callbacks,
     verbose=2)

Epoch 1/50
225/225 - 4s - loss: 0.5717 - accuracy: 0.7136 - val_loss: 0.4940 - val_accuracy: 0.7788
Epoch 2/50
225/225 - 4s - loss: 0.3603 - accuracy: 0.8643 - val_loss: 0.4092 - val_accuracy: 0.8208
Epoch 3/50
225/225 - 4s - loss: 0.2909 - accuracy: 0.8874 - val_loss: 0.3597 - val_accuracy: 0.8468
Epoch 4/50
225/225 - 4s - loss: 0.2593 - accuracy: 0.9002 - val_loss: 0.3322 - val_accuracy: 0.8628
Epoch 5/50
225/225 - 4s - loss: 0.2401 - accuracy: 0.9076 - val_loss: 0.3106 - val_accuracy: 0.8748
Epoch 6/50
225/225 - 4s - loss: 0.2279 - accuracy: 0.9129 - val_loss: 0.3365 - val_accuracy: 0.8608
Epoch 7/50
225/225 - 4s - loss: 0.2193 - accuracy: 0.9162 - val_loss: 0.3078 - val_accuracy: 0.8780
Epoch 8/50
225/225 - 4s - loss: 0.2118 - accuracy: 0.9198 - val_loss: 0.3487 - val_accuracy: 0.8592
Epoch 9/50
225/225 - 4s - loss: 0.2065 - accuracy: 0.9230 - val_loss: 0.3446 - val_accuracy: 0.8608
Epoch 10/50
225/225 - 4s - loss: 0.2016 - accuracy: 0.9236 - val_loss: 0.3435 - val_accuracy: 0.8656

In [10]:
model.evaluate(x_test_pad, y_test)

782/782 [==============================] - 3s 4ms/step - loss: 0.2908 - accuracy: 0.8824


[0.2907903790473938, 0.8823999762535095]